In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaModel, RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from PolymerSmilesTokenization import PolymerSmilesTokenizer
from dataset import Downstream_Dataset, DataAugmentation, LoadPretrainData
import torch
import torch.nn as nn
from torchmetrics import R2Score
from torch.utils.tensorboard import SummaryWriter
from copy import deepcopy
from torch.utils.data import Dataset, DataLoader


C:\Python\Python310\lib\site-packages\transformers\trainer_pt_utils.py:211: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  device: Optional[torch.device] = torch.device("cuda"),


Downstream_Dataset class is called


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data = pd.read_csv('data/practice.csv')
len(train_data)


10

In [3]:
scaler = StandardScaler()
train_data.iloc[:, 1] = scaler.fit_transform(train_data.iloc[:, 1].values.reshape(-1, 1))
train_data

,smiles,value
0,*C*,0.671870
1,*CC(*)C,0.440891
2,*CC(*)CC,0.439301
3,*CC(*)CCC,0.571796
4,*CC(*)CC(C)C,0.575343
5,*CC1CCC(*)C1,0.711080
6,*CC(*)CCCC1CCCCC1,0.349564
7,*C=CCCC*,-0.710518
8,*C=CCC*,-0.358177
9,*C=C*,-2.691151


In [15]:
tokenizer = PolymerSmilesTokenizer.from_pretrained("roberta-base", max_len=411)
text = train_data['smiles'][6]
print(text)
tokens = tokenizer.tokenize(text)
print(tokens)
train_dataset = Downstream_Dataset(train_data, tokenizer, 411)
train_dataset[6]['input_ids']


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'PolymerSmilesTokenizer'.


*CC(*)CCCC1CCCCC1
['*', 'C', 'C', '(', '*', ')', 'C', 'C', 'C', 'C', '1', 'C', 'C', 'C', 'C', 'C', '1']


tensor([   0, 3226,  347,  347, 1640, 3226,   43,  347,  347,  347,  347,  134,
         347,  347,  347,  347,  347,  134,    2,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,   

In [5]:

config = RobertaConfig(
            vocab_size=50265,
            max_position_embeddings=514,
            num_attention_heads=12,
            num_hidden_layers=6,
            type_vocab_size=1,
            hidden_dropout_prob=0.1,
            attention_probs_dropout_prob=0.1
        )
PretrainedModel = RobertaModel(config=config)
embeddings = PretrainedModel.embeddings.word_embeddings(train_dataset[6]['input_ids'])
print(embeddings)
print(embeddings.size())
print(embeddings[0])

tensor([[-0.0010,  0.0193,  0.0198,  ...,  0.0112, -0.0060, -0.0348],
        [ 0.0221,  0.0339, -0.0025,  ...,  0.0096, -0.0119,  0.0099],
        [ 0.0002, -0.0194,  0.0157,  ..., -0.0070, -0.0220,  0.0310],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([411, 768])
tensor([-9.7080e-04,  1.9268e-02,  1.9810e-02, -5.1340e-03, -6.2634e-03,
        -2.7622e-04, -8.4029e-04,  1.1478e-02, -2.7793e-02, -3.1914e-02,
         1.0053e-02, -4.8070e-03, -2.0411e-02,  1.0643e-02,  5.7324e-05,
         2.6326e-02, -1.1406e-02,  1.8332e-02, -1.2637e-02,  2.0174e-02,
        -2.6650e-02, -1.3453e-02, -1.0993e-02, -1.1852e-02, -1.3301e-02,
         8.3330e-04,  4.9652e-02, -1.0200e-02, -3.1332e-03,  1.3913e-02,
         2.7729e-02, -5.9845e-03,  1.6797e-02, -8.2463e-03, -1.3446e

In [6]:
class DownstreamRegression(nn.Module):
    print('Class DownstreamRegression is called')
    def __init__(self, drop_rate=0.1):
        super(DownstreamRegression, self).__init__()
        self.PretrainedModel = deepcopy(PretrainedModel)
        self.PretrainedModel.resize_token_embeddings(len(tokenizer))
        
        self.Regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(self.PretrainedModel.config.hidden_size, self.PretrainedModel.config.hidden_size),
            nn.SiLU(),
            nn.Linear(self.PretrainedModel.config.hidden_size, 1)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.PretrainedModel(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.last_hidden_state[:, 0, :] #fingerprint
        print(f'Finger print is:')
        print(logits)
        print(logits.size())
        output = self.Regressor(logits)
        return output

Class DownstreamRegression is called


In [7]:
def train(model, optimizer, scheduler, loss_fn, train_dataloader, device):
    print('Train func is called')
    model.train()

    for step, batch in enumerate(train_dataloader):
        print(f'Batch and Step: {step}')
        input_ids = batch["input_ids"].to(device)
        print('Batch')
        print(input_ids)
        attention_mask = batch["attention_mask"].to(device)
        prop = batch["prop"].to(device).float()
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask).float()
        print('Output for step',step)
        print(outputs)
        loss = loss_fn(outputs.squeeze(), prop.squeeze())
        loss.backward()
        optimizer.step()
        scheduler.step()
        print('End of one step')
        print('--------------------------------------------------------------------------------')

    return None

In [8]:
model = DownstreamRegression(drop_rate=0.1).to(device)
model = model.double()
loss_fn = nn.MSELoss()

In [9]:
train_dataloader = DataLoader(train_dataset, 1, shuffle=True, num_workers=8)

In [10]:
"""Parameters for scheduler"""
steps_per_epoch = train_data.shape[0] // 1
training_steps = steps_per_epoch * 1
warmup_steps = int(training_steps * 0.05)

In [11]:
optimizer = AdamW(
                    [
                        {"params": model.PretrainedModel.parameters(), "lr":  0.00005,
                         "weight_decay": 0.0},
                        {"params": model.Regressor.parameters(), "lr": 0.0001,
                         "weight_decay": 0.01},
                    ],
    				no_deprecation_warning=True
                )
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                                        num_training_steps=training_steps)

In [12]:
for epoch in range(1):
    print("Training epoch: %s/%s" % (epoch+1, 1))
    train(model, optimizer, scheduler, loss_fn, train_dataloader, device)
    print('End of one Epoch')
    print('|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||')

Training epoch: 1/1
Train func is called


KeyboardInterrupt: 

# Pretraining 

In [ ]:
def split(file_path):
    dataset = pd.read_csv(file_path, header=None).values
    train_data, valid_data = train_test_split(dataset, test_size=0.2, random_state=1)
    return train_data, valid_data

config = RobertaConfig(
        vocab_size=50265,
        max_position_embeddings=514,
        num_attention_heads=12,
        num_hidden_layers=6,
        type_vocab_size=1,
        hidden_dropout_prob=0.1,
        attention_probs_dropout_prob=0.1,
    )

tokenizer = PolymerSmilesTokenizer.from_pretrained("roberta-base", max_len=175)
model = RobertaForMaskedLM(config=config).to(device)

In [ ]:
train_data, valid_data = split('data/P1_practice.csv')
data_train = LoadPretrainData(tokenizer=tokenizer, dataset=train_data, blocksize=175)
data_valid = LoadPretrainData(tokenizer=tokenizer, dataset=valid_data, blocksize=175)
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.15 
    )
text = train_data[3][0]
print(text)
tokens = tokenizer.tokenize(text)
print(tokens)
print(data_train[3]['input_ids'])
print(data_train[3]['input_ids'].size())

In [ ]:
embeddings = model.roberta.embeddings.word_embeddings(data_train[3]['input_ids'])
print(embeddings)
print(embeddings.size())
print(embeddings[0])